In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas
from sklearn.utils import shuffle
import gensim
import numpy as np

In [3]:
raw_data = pandas.DataFrame.from_csv(path='../data/people_attributes.csv', index_col=None)
#raw_data = raw_data.set_index('name', drop=False)
raw_data.head()

,name,occupation,description,gender
0,Barack Obama,Politician,44th President of the United States of America,male
1,Barbara Boxer,Politician,American politician,female
2,Bill Clinton,Politician,42nd President of the United States,male
3,Carly Fiorina,Politician,American corporate executive and politician,female
4,Clint Eastwood,Actor,actor and director from the United States,male


In [5]:
name_and_occ = raw_data.drop(['gender', 'description'], axis=1)
name_and_occ = shuffle(name_and_occ)
name_and_occ.head()

,name,occupation
165,Frederick W. Smith,Businessperson
146,Muhtar Kent,Businessperson
177,Toxey Haas,Businessperson
69,Al Franken,Politician
277,Lon R. Greenberg,Businessperson


In [6]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True) 